In [ ]:
import nltk
from nltk.corpus import stopwords, movie_reviews
from nltk.tokenize import word_tokenize
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
nltk.download('stopwords')
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\U765123\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\U765123\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\U765123\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [27]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)
stop_words = set(stopwords.words('english'))


In [28]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(tokens)

In [29]:
documents = [(preprocess_text(' '.join(doc)), category) for doc, category in documents]

In [30]:
# Separating the data and labels
texts, labels = zip(*documents)

In [31]:
# this is a bag of words cassifier
vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(texts)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=42)


In [33]:
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [34]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.60%


In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.84      0.86       254
         pos       0.84      0.87      0.86       246

    accuracy                           0.86       500
   macro avg       0.86      0.86      0.86       500
weighted avg       0.86      0.86      0.86       500



In [37]:
# Function to predict sentiment
def predict_sentiment(text):
    preprocessed_text = preprocess_text(text)
    text_features = vectorizer.transform([preprocessed_text])
    sentiment = model.predict(text_features)
    return sentiment[0]

# Test the function with made up sample text
sample_text = "This movie was kind of okay but also not a big fan of it."
print(f"Sentiment: {predict_sentiment(sample_text)}")

Sentiment: neg
